In [5]:
import numpy as np
import glob
import tensorflow.compat.v1 as tf
from sklearn.metrics.pairwise import euclidean_distances
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from munkres import Munkres
k_cluster = 38
batchSize=486

x_train=np.load('../../datasets/eyaleb/d_eyalebtrain.npy')
x_train=x_train.reshape(len(x_train)*batchSize,784)
y=np.load('../../datasets/eyaleb/l_eyalebtrain.npy')
y=y.reshape(len(y)*batchSize,)
alpha_trans1=np.load('../../datasets/eyaleb/h_eyalebtrain.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 

x=np.load('../../datasets/eyaleb/d_eyaleb.npy')
x=x.reshape(len(x)*batchSize,784)
x_test=x[len(x_train):,:]
y1=np.load('../../datasets/eyaleb/l_eyaleb.npy')
y1=y1.reshape(len(y1)*batchSize,)
y_test=y1[len(y):,]
y_test=y_test.reshape(len(y_test),)

In [6]:
def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    for a in range (1):
        if uc1.shape==uc2.shape:
            m = np.ones([l1, l2])
            for i in range(l1):
                it_i = np.nonzero(c1 == uc1[i])[0]
                for j in range(l2):
                    it_j = np.nonzero(c2 == uc2[j])[0]
                    m_ij = np.intersect1d(it_j, it_i)
                    m[i,j] =  -m_ij.size
        else:
            print('assertion handeled')
            return
        #break
    return m


def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes)
    if cost_matrix is None:
        return (None,None)
    else:
        m = Munkres()
        indexes = m.compute(cost_matrix)
        mapper = { old: new for (old, new) in indexes }
        new_labels = translate_clustering(labels, mapper)
        new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
        return new_labels,accuracy(new_cm)

def arch():
    encoding_dim = 484 
    input_img = Input(shape=(784,))
    encoded_layer_2 = Dense(700, activation='tanh')(input_img)
    encoded_layer_3 = Dense(600, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(500, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(500, activation='tanh')(encoded)
    decoded_layer_3 = Dense(600, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(700, activation='tanh')(decoded_layer_3)
    decoded = Dense(784, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder
  
def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T         
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

In [7]:
encoder,encoding_dim,autoencoder=arch()
encoder.load_weights('../../checkpoints/eyaleb/sdsc_eyalebtrain.ckpt')
z_test=encoder.predict(x_test)
ic= np.load('../../checkpoints/eyaleb/sdsc_eyalebictrain.npy')
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y_test)
nmi=sklearn.metrics.normalized_mutual_info_score(y_test, rectified_label)
f1=sklearn.metrics.f1_score(y_test, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y_test, rectified_label,average='weighted')
print('Results on eyaleb test dataset: accuracy is, nmi is, f1 score is, precision is...',acc,nmi,f1,prec)

Results on eyaleb test dataset: accuracy is, nmi is, f1 score is, precision is... 1.0 1.0 1.0 1.0
